In [ ]:
from langchain_core.documents import Document

document = Document("안녕하세요? 이건 랭체인의 도큐먼트 입니다.")

# 도큐먼트 속성 확인
document.__dict__

# metadata 추가
document.metadata["source"] = "TeddyNote"
document.metadata["page"] = 1
document.metadata["author"] = "Teddy"

In [11]:
from langchain_community.document_loaders import PyPDFLoader, CSVLoader

FILE_PATH = "./test_data.csv"

# Document Loader 
# 불러온 내용을 "문서 객체"로 변환하는 역할

# 로더 설정
loader = CSVLoader(FILE_PATH)
docs = loader.load()

# 로드된 문서의 수 확인
print(len(docs))
print(docs[0].metadata)


17
{'source': './test_data.csv', 'row': 0}


In [25]:
from langchain_text_splitters import CharacterTextSplitter


# CSV 파일 경로
loader = CSVLoader(
    file_path = "./test_data.csv",
    csv_args={
        "delimiter": ",",   # 구분자
        "quotechar": '"',   # 인용 부호 문자
        "fieldnames": [     # 필드 이름
            "region",
            "prc",
            "rentPrc",
            "spc1",
            "spc2",
            "prc/spc1",
            "rentPrc/spc1",
            "prc/spc2",
            "rentPrc/spc2"
        ]
    }
)


# CSV로더 설정, 파일 경로 및 소스 칼럼 지정
loader = CSVLoader(
    file_path='./test_data.csv', source_column='prc'
)

docs = loader.load()

print(docs[1])

page_content='﻿region: 금호2가동
prc: 18457.894736842107
rentPrc: 170.0
spc1: 84.83333333333333
spc2: 62.46947368421052
prc/spc1: 217.57832695688143
rentPrc/spc1: 2.0039292730844793
prc/spc2: 295.47062986553436
rentPrc/spc2: 2.721329154450174' metadata={'source': '18457.894736842107', 'row': 1}


In [ ]:
from langchain_community.document_loaders.csv_loader import UnstructuredCSVLoader, CSVLoader

import pandas as pd

# 비구조화 CSV 로더 인스턴스 생성
# loader = UnstructuredCSVLoader(file_path="./test_estate.csv", mode="elements")
loader = CSVLoader(file_path="../../estate_data/naver_region_name_estate.csv", encoding='utf-8-sig')

docs = loader.load()

print(len(docs))
print(docs[0])

2426
page_content='atclNo: 2538700418
atclNm: 서울숲아이파크리버포레1차
region: 성수1가제2동
rletTpNm: 아파트
tradTpNm: 월세
flrInfo: 23/33
prc: 60000
rentPrc: 140
spc1: 81
spc2: 59.94
direction: 남서향
atclCfmYmd: 25.07.21.
lat: 37.551266
lng: 127.043521
atclFetrDesc: 지상철 소음없는 조용한동 선점 기회
bildNm: 104동
rltrNm: 청담워니공인중개사사무소' metadata={'source': '../../estate_data/naver_region_name_estate.csv', 'row': 0}


In [55]:
# After data loading and before data embedding, i should check data chunking.
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

# text_splitter = CharacterTextSplitter(
#     separator = '\n',
#     chunk_size = 500,
#     chunk_overlap = 5,
#     length_function = len    
# )

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 400,
    chunk_overlap =15,
    length_function = len    
)

text = text_splitter.split_documents(docs)
print(f"분할된 청크의 수 : {len(text)}")
print(text[1])

# print(len(text))

분할된 청크의 수 : 2426
page_content='atclNo: 2538391119
atclNm: 서울숲아이파크리버포레1차
region: 성수1가제2동
rletTpNm: 아파트
tradTpNm: 월세
flrInfo: 28/33
prc: 60000
rentPrc: 375
spc1: 115
spc2: 84.95
direction: 남향
atclCfmYmd: 25.07.17.
lat: 37.551107
lng: 127.042365
atclFetrDesc: 84, 고층 남향, 보증금 유연한 변동 가능
bildNm: 103동
rltrNm: 아이파크(단지내)YES공인중개사사무소' metadata={'source': '../../estate_data/naver_region_name_estate.csv', 'row': 1}


In [ ]:
# load .csv file embedding practice

''' **** 기본 teddynote rag_project 전체 개요에서 사용한 임베딩 과정 ****
# 단계 3 : 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 단계 4 : DB 생성(Create DB) 및 저장
# 벡터 스토어를 생성합니다.
# !pip install faiss-cpu
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)
'''

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# embeddings_model = OpenAIEmbeddings()

# embeddings = embeddings_model.embed_documents()

# 단계 3 : 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 단계 4 : DB 생성(Create DB) 및 저장
# 벡터 스토어를 생성합니다.
# !pip install faiss-cpu
vectorstore = FAISS.from_documents(documents=text, embedding=embeddings)

[Document(id='6a4847ff-1b0b-4a03-9b40-9bd03f27595d', metadata={'source': '../../estate_data/naver_region_name_estate.csv', 'row': 769}, page_content='atclNo: 2539384948\natclNm: 일반원룸\nregion: 행당제2동\nrletTpNm: 원룸\ntradTpNm: 월세\nflrInfo: 5/5\nprc: 3000\nrentPrc: 45\nspc1: -\nspc2: 16.52\ndirection: 북서향\natclCfmYmd: 25.07.21.\nlat: 37.560466\nlng: 127.043284\natclFetrDesc: 월세저렴 풀옵션원룸 한양대인근 조용한주택가 보증금조절가능\nbildNm: \nrltrNm: 더드림부동산'),
 Document(id='91295993-f0fd-4f5d-9361-c34650067b5f', metadata={'source': '../../estate_data/naver_region_name_estate.csv', 'row': 1664}, page_content='atclNo: 2534647685\natclNm: 도시형생활주택\nregion: 응봉동\nrletTpNm: 원룸\ntradTpNm: 월세\nflrInfo: 3/5\nprc: 3000\nrentPrc: 45\nspc1: -\nspc2: 14.08\ndirection: 북향\natclCfmYmd: 25.06.27.\nlat: 37.556521\nlng: 127.041602\natclFetrDesc: 한양대 정문 도보 1분, 구조 좋고 깨끗, 주택가 위치\nbildNm: \nrltrNm: 봄날공인중개사사무소'),
 Document(id='4acc8e57-da0d-4d8f-bad5-6e71673eb732', metadata={'source': '../../estate_data/naver_region_name_estate.csv', 'row'

In [ ]:
# 단계 5 : 검색기 (Retriever) 생성

# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever(search_kwargs={'k': 30})

# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("월세 60이하, 6평 이상, 그리고 한양대학교와의 거리를 고려해서 월세 방을 5곳 추천해줘." \
"추천한 이유와 추천된 곳의 매물 이름, 월세, 보증금, 중개하는 부동산 이름을 같이 알려줘.")

In [58]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 단계 6 : 프롬프트 생성(Create Prompt)

prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks.
    User the following peices of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Anser in Korean.
    retrieved context includes information of Deposit(column anme 'prc'), Monthly Rent(column name 'rentPrc'),
    Total Floor Area(in Korean, 공급면적. column name 'spc1'), Exclusive Use Area(in Korean, 전용면적. column name 'spc2'),
    Listing type(or Housing type. It includes 'apartment, studio aparmtent, officetel etc'. column name 'rletTpNm').
    District(column name 'region)'.
    All data come from Real Estat Listing Website. And I extract about Seongdong-Gu list, which is closed to Hanyang University, Korea.
    
    #Question:
    {question}

    #Context:
    {context}

    #Answer:
    """    
)

# 단계 7 : 언어모델(LLM) 생성

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 단계 8 : 체인(chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm    
    | StrOutputParser()
)

# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "월세 60이하, 6평 이상, 그리고 한양대학교와의 거리를 고려해서 월세 방을 5곳 추천해줘." \
"추천한 이유와 추천된 곳의 매물 이름, 월세, 보증금, 중개하는 부동산 이름을 같이 알려줘."
response = chain.invoke(question)
print(response)

다음은 월세 60 이하, 6평 이상, 한양대학교와 가까운 월세 방 5곳 추천입니다:

1. **매물 이름**: 일반원룸
   - **월세**: 45만 원
   - **보증금**: 3000만 원
   - **중개 부동산 이름**: 더드림부동산
   - **추천 이유**: 한양대 인근에 위치하며 조용한 주택가에 있어 생활하기 좋습니다. 보증금 조절이 가능하다는 점도 장점입니다.

2. **매물 이름**: 도시형생활주택
   - **월세**: 45만 원
   - **보증금**: 3000만 원
   - **중개 부동산 이름**: 봄날공인중개사사무소
   - **추천 이유**: 한양대 정문에서 도보 1분 거리로 매우 가깝고, 구조가 좋고 깨끗한 주택가에 위치해 있습니다.

3. **매물 이름**: 일반원룸
   - **월세**: 30만 원
   - **보증금**: 6000만 원
   - **중개 부동산 이름**: 봄날공인중개사사무소
   - **추천 이유**: 한양대 도보 1분 거리로 매우 가깝고, 보증금 조정이 가능하며 구조가 좋습니다.

4. **매물 이름**: 일반원룸
   - **월세**: 45만 원
   - **보증금**: 3000만 원
   - **중개 부동산 이름**: 더드림부동산
   - **추천 이유**: 한양대 인근에 위치하며 조용한 주택가에 있어 생활하기 좋습니다. 풀옵션으로 제공됩니다.

5. **매물 이름**: 일반원룸
   - **월세**: 45만 원
   - **보증금**: 3000만 원
   - **중개 부동산 이름**: 봄날공인중개사사무소
   - **추천 이유**: 한양대 정문에서 도보 1분 거리로 매우 가깝고, 구조가 좋고 깨끗한 주택가에 위치해 있습니다.

이 매물들은 모두 한양대학교와 가까운 거리에 위치해 있어 통학에 유리하며, 월세와 보증금 조건도 비교적 합리적입니다.
